In [1]:
import pandas as pd

data = pd.read_csv('/kaggle/input/video-game-sales-2024/vgchartz-2024.csv')
data.head()

,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


In [2]:
data = data.drop(columns=["img","title","console","publisher","developer","critic_score","na_sales","jp_sales","release_date","last_update"])
data

,genre,total_sales,pal_sales,other_sales
0,Action,20.32,9.85,3.12
1,Action,19.39,9.71,3.02
2,Action,16.15,5.49,1.78
3,Action,15.86,5.33,1.42
4,Shooter,15.09,6.05,2.44
...,...,...,...,...
64011,Visual Novel,NaN,NaN,NaN
64012,Visual Novel,NaN,NaN,NaN
64013,Visual Novel,NaN,NaN,NaN
64014,Visual Novel,NaN,NaN,NaN


In [3]:
data = data.dropna()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12648 entries, 0 to 18920
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   genre        12648 non-null  object 
 1   total_sales  12648 non-null  float64
 2   pal_sales    12648 non-null  float64
 3   other_sales  12648 non-null  float64
dtypes: float64(3), object(1)
memory usage: 494.1+ KB


In [5]:
data['genre'].value_counts()

genre
Action              2112
Sports              1860
Shooter             1256
Racing              1209
Misc                1164
Adventure            902
Platform             795
Role-Playing         737
Simulation           691
Fighting             568
Strategy             502
Puzzle               494
Action-Adventure     219
Music                 90
MMO                   19
Party                 17
Visual Novel           8
Education              3
Sandbox                1
Board Game             1
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2) 

len (data), len(train), len(test)

(12648, 10118, 2530)

In [7]:
train_data, y_train = train.drop(['total_sales'], axis=1), train['total_sales'].copy()
test_data, y_test = test.drop(['total_sales'], axis=1), test['total_sales'].copy()

train_num = train_data.drop(['genre'], axis=1)
train_cat = train_data[['genre']]

In [8]:
y_test

2225     0.72
9948     0.11
2884     0.57
606      1.81
2727     0.60
         ... 
6340     0.24
462      2.15
5358     0.29
99       4.79
10097    0.10
Name: total_sales, Length: 2530, dtype: float64

In [9]:
train_data

,genre,pal_sales,other_sales
1851,Shooter,0.27,0.13
1957,Role-Playing,0.10,0.05
5019,Puzzle,0.02,0.02
11166,Racing,0.06,0.01
8722,Fighting,0.03,0.00
...,...,...,...
4698,Misc,0.08,0.01
17141,Adventure,0.01,0.00
7353,Sports,0.05,0.00
6215,Racing,0.03,0.03


In [10]:
y_train

1851     0.85
1957     0.81
5019     0.32
11166    0.08
8722     0.14
         ... 
4698     0.35
17141    0.01
7353     0.19
6215     0.24
4479     0.37
Name: total_sales, Length: 10118, dtype: float64

In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median") 
imputer.fit(train_num) 
imputer.statistics_

array([0.04, 0.01])

In [12]:
X_train_num = imputer.transform(train_num) 

X_train_num

array([[0.27, 0.13],
       [0.1 , 0.05],
       [0.02, 0.02],
       ...,
       [0.05, 0.  ],
       [0.03, 0.03],
       [0.2 , 0.04]])

In [13]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler() # mean y std
scaler.fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
X_train_num_scaled

array([[ 0.29900357,  0.56332058],
       [-0.1261852 , -0.00179426],
       [-0.32627403, -0.21371232],
       ...,
       [-0.25124072, -0.35499103],
       [-0.30126293, -0.14307297],
       [ 0.12392584, -0.07243361]])

In [14]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
X_train_cat = cat_encoder.fit_transform(train_cat)
X_train_cat

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10118 stored elements and shape (10118, 20)>

In [15]:
X_train_cat.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
y_train

1851     0.85
1957     0.81
5019     0.32
11166    0.08
8722     0.14
         ... 
4698     0.35
17141    0.01
7353     0.19
6215     0.24
4479     0.37
Name: total_sales, Length: 10118, dtype: float64

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(train_num)
cat_attribs = ["genre"]

full_pipeline = ColumnTransformer([
	("num", num_pipeline, num_attribs),
	("cat", OneHotEncoder(), cat_attribs),
])

In [18]:
X_train = full_pipeline.fit_transform(train_data)

X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 30354 stored elements and shape (10118, 22)>

In [19]:
X_test = full_pipeline.transform(test_data) 

X_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7590 stored elements and shape (2530, 22)>

In [20]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [21]:
preds = lin_reg.predict(X_test)
preds[:10]

array([0.74323525, 0.17191002, 0.6054358 , 1.04042366, 0.52108142,
       0.08254894, 0.08174439, 0.15518141, 0.2373796 , 0.25545985])